In [1]:
import sys
import os
import cvxpy as cp
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/core")

os.environ['MOSEKLM_LICENSE_FILE'] = module_path

import core
import mosek
from core.controllers import QPController
from core.dynamics import AffineDynamics, AffineQuadCLF, ScalarDynamics, SystemDynamics
from core.util import arr_map
from core.controllers import FBLinController, LQRController
from core.systems import InvertedPendulum

In [16]:

def simulate(system, controller, x_0, T, num_steps):
    ts = np.linspace(0, T, num_steps)
    xs, us = system.simulate(x_0, controller, ts)
    print(xs.shape,us.shape)
    return xs, us, ts

def build_ccf_data(aff_lyap,xs,us,ts):
    'computing the error in the derivative of lyapunov function using finite differencing'
    # need batch computation in image and forward
    # zs = aff_lyap.eval(xs[2:],ts[2:]) - aff_lyap.eval(xs[:-2],ts[:-2]) - \
    #     aff_lyap.eval_dot(xs[1:-1],us[1:-1],ts[1:-1])
    
    #z0 = [aff_lyap.eval(x,t) for x,t in zip(xs,ts)]
    #z1 = [(aff_lyap.eval(xs[i+1],ts[i+1])- aff_lyap.eval(xs[i-1],ts[i-1]))/2 for i in range(1, len(ts)-1)]
    av_x = (xs[:-1] + xs[1:])/2
    z2 = [aff_lyap.eval_dot(x,u,t) for x,u,t in zip(xs[1:-1],us[1:],ts[1:-1])]
    zs= [aff_lyap.eval(xs[i+1],ts[i+1])- aff_lyap.eval(xs[i],ts[i]) - \
        aff_lyap.eval_dot(av_x[i],us[i],ts[i]) for i in range(len(ts)-1)]
    print(z2)
    print(zs)
    us = us[1:]
    ys = np.concatenate((np.ones((len(us),1)), us), axis=1)
    return xs[1:-1], ys, np.array(zs)
    # zs= [(aff_lyap.eval(x+1,t+1)- aff_lyap.eval(x,t)) - \
    #     aff_lyap.eval_dot(x,u,t) for x,u,t in zip(xs[:-1],us,ts[:-1])]
    # ys = np.concatenate((np.ones((len(us),1)), us), axis=1)
    # return xs[:-1], ys, np.array(zs)


def main():
    system = InvertedPendulum(0.7,0.7)
    system_est = InvertedPendulum(0.50,0.5)
    lyap = AffineQuadCLF.build_care(system, Q=np.identity(2), R=np.identity(1))
    lyap_est = AffineQuadCLF.build_care(system_est, Q=np.identity(2), R=np.identity(1))
    
    alpha = 1 / max(la.eigvals(lyap_est.P))

    # Linearizing Controller 
    controller_linear = FBLinController(system_est, LQRController(system_est, lyap.P, R=np.identity(1)))
    
    # Nominal Controller Static Slacked
    controller = QPController(system_est, system.m)
    controller.add_static_cost(np.identity(1))
    controller.add_stability_constraint(lyap_est, comp=lambda r: alpha * r, slacked=True, coeff = 1e3)

    x_0 = np.array([2, 0.1])
    xs = np.linspace(-5, 5, 100)
    us = np.linspace(-1, 1, 100)
    ts = np.linspace(0, 2, 100)
    xs, us, ts = simulate(system, controller, x_0, 2, 200)
    xs, ys, zs = build_ccf_data(lyap_est,xs,us,ts) #ts=ts[1:-1]
    #print(ys.shape,zs)
    


    

In [17]:
main()

(array([-6.6865086]), [array(0.01230718)]) 0
(array([-6.43253593]), [array(0.01204854)]) 1
(array([-6.18387691]), [array(0.01178345)]) 2
(array([-5.94063943]), [array(0.01151241)]) 3
(array([-5.702926]), [array(0.01123596)]) 4
(array([-5.47083278]), [array(0.01095468)]) 5
(array([-5.24444859]), [array(0.01066916)]) 6
(array([-5.02385401]), [array(0.01038005)]) 7
(array([-4.80912046]), [array(0.01008798)]) 8
(array([-4.60030948]), [array(0.00979365)]) 9
(array([-4.39747193]), [array(0.00949773)]) 10
(array([-4.20064735]), [array(0.00920091)]) 11
(array([-4.00986351]), [array(0.00890388)]) 12
(array([-3.8251359]), [array(0.00860734)]) 13
(array([-3.6464675]), [array(0.00831196)]) 14
(array([-3.47384857]), [array(0.00801839)]) 15
(array([-3.30725665]), [array(0.00772728)]) 16
(array([-3.14665661]), [array(0.00743924)]) 17
(array([-2.99200087]), [array(0.00715486)]) 18
(array([-2.84322976]), [array(0.00687468)]) 19
(array([-2.7002719]), [array(0.00659921)]) 20
(array([-2.5630448]), [array(